In supermarkets, loose fruits and vegetables need to be weighted to determine te price to pay. In some supermarkets weighing is done by workers, in others it's done by the customers themselves. To avoid fraud or reduce the number of employees dedicated to this task (which is tedious and costly), we propose a system that would automatically recognize the type of fruit or vegetable while weighting. Such a system would also be useful to have a more up-to-date information on stocks in the section. 

The goal of this RAMP is to provide a proof of concept of the system and to assess its feasibility in a simpler case before more sophisticated developement.

image cool a insérer

For this POC, the number of classes has been limited to 5 types of fruits () and 5 types of vegetables (). We used an API from Bing :  nom de l'API. How does it work? To verify the quality of the dataset, we checked it manually and removed errors. That was time consumming, therfore we had to limit the size of dataset to X images by class. As the background enviromnent is a supermarket, images without background are not enough. It was important to have a dataset both with and without backgrounds.

The goal of this system is to determine the class of the product to determine its price (per kg). If the system predicts a wrong class, the price will be inacurate. If the price price is lower than the real one, companies will loose money, if it higher clients will be unsatisfied and claim for a discount. The loss of income can therefore we direclty related to the accuracy of the algorithm. As the goal of the system is to replace employees or customers, it needs to have a high accuracy before being released. For the moment, we have an accuracy of X % for our POC, we would like to have an accuracy of at least Y % to validate this Poc and go further in experimentation.

If this POC is validated, we could :
- extend this system to classify more types of fruitsn vegetables, and other loose goods
- take into account multiple varieties for one class (beafsteak tomatos, cherry tomatos, cold & green tomatos,...)


We should keep in mind that this is only a Poc so the case is a simpler case. Developing these extensions would requiere to overcome new difficulties : much more larger dataset, unbalanced dataset...
These difficulties could be addressed in a new challenge !



Liens qui être utiles/ présentent pourquoi le sujet est interessant :

https://ac.els-cdn.com/S016816990900180X/1-s2.0-S016816990900180X-main.pdf?_tid=59001dbe-a574-41ca-be6d-82c8e1861665&acdnat=1548153802_e2ac8effa1d33d978a64b537f858468f

https://github.com/Horea94/Fruit-Images-Dataset

https://www.cs.toronto.edu/~kriz/cifar.html (cifar Dataset : contient un subdataset fuit and vegetable)

RobustApproachforFruitandVegetableClassification (article PDF)



In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'dataset_v1/train'
validation_data_dir = 'dataset_v1/validation'
nb_classes = 10
nb_train_samples = 400*10
nb_validation_samples = 100*10
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Ceci est l'augmentation de données utilisée pour le training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True)

# Ceci est l'augmentation utilisée pour le testing, on ne fait que rescaler
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Using TensorFlow backend.


Found 1782 images belonging to 10 classes.
Found 1782 images belonging to 10 classes.
Epoch 1/50
 48/125 [==========>...................] - ETA: 2:09 - loss: 2.2056 - acc: 0.2747

/home/louis/Documents/3A/Datacamp_team_project/env_datacamp/lib/python3.6/site-packages/PIL/Image.py:969: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


104/125 [=======================>......] - ETA: 35s - loss: 2.0340 - acc: 0.3173

/home/louis/Documents/3A/Datacamp_team_project/env_datacamp/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


125/125 [==============================] - 271s 2s/step - loss: 2.0036 - acc: 0.3263 - val_loss: 1.9169 - val_acc: 0.3350
Epoch 2/50
125/125 [==============================] - 268s 2s/step - loss: 1.7150 - acc: 0.4185 - val_loss: 1.5991 - val_acc: 0.4400
Epoch 3/50
124/125 [============================>.] - ETA: 1s - loss: 1.5796 - acc: 0.4568

KeyboardInterrupt: 